# <small>FACIAL EMOTION</small><br><font color="blueviolet">RECOGNITION</font>

In [1]:
from pandas import read_csv as csv
from numpy import array, mean, std, expand_dims, argmax
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing import image
from keras.utils import np_utils
from keras.models import model_from_json
from imutils import resize
import cv2

Using TensorFlow backend.


In [2]:
df = csv('dataset/fer2013.csv') # load pixel-based dataset
print(df.info()) # display dataset info
df # display dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [3]:
num_features, num_labels = 64, 7
width, height = 48, 48

In [4]:
x_train, y_train, x_test, y_test = [], [], [], []
for index, row in df.iterrows():
    val = row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            x_train.append(array(val,'float32'))
            y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            x_test.append(array(val,'float32'))
            y_test.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")
        
# wrap them all in a single array
x_train = array(x_train, 'float32')
y_train = array(y_train, 'float32')
x_test = array(x_test, 'float32')
y_test = array(y_test, 'float32')

# label y == categorical data
y_train = np_utils.to_categorical(y_train, num_classes=num_labels)
y_test = np_utils.to_categorical(y_test, num_classes=num_labels)

# normalization
x_train -= mean(x_train, axis=0)
x_train /= std(x_train, axis=0)
x_test -= mean(x_test, axis=0)
x_test /= std(x_test, axis=0)

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
print("x_train {} y_train {}".format(x_train.shape, y_train.shape))
print("x_test {} y_test {}".format(x_test.shape, y_test.shape))

x_train (28709, 48, 48, 1) y_train (28709, 7)
x_test (3589, 48, 48, 1) y_test (3589, 7)


In [5]:
model = Sequential([
    # 1st convolution layer
    Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(x_train.shape[1:])),
    Conv2D(64,kernel_size= (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Dropout(.5),

    # 2nd convolution layer
    Conv2D(64, (3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2), strides=(2, 2)),
    Dropout(.5),

    # 3rd convolution layer
    Conv2D(128, (3, 3), activation='relu'),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2), strides=(2, 2)),
    
    # fully connected neural networks
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    Dropout(.2),
    Dense(num_labels, activation='softmax')
], name="Facial_Emotion_Recognition")
model.summary()

Model: "Facial_Emotion_Recognition"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 44, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 18, 

In [6]:
# model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
# model.fit(x=x_train, y=y_train, epochs=200, batch_size=1, shuffle=True, validation_data=(x_test, y_test))

In [7]:
# # save model weights
# fer_json = model.to_json()
# with open("model/fer.json", "w") as json_file:
#     json_file.write(fer_json)
# model.save_weights("model/fer.h5")

In [8]:
# reload the saved model
model = model_from_json(open("model/fer.json", "r").read())
model.load_weights('model/fer.h5')

def start_camera():
    face_haar_cascade = cv2.CascadeClassifier("E:\DATA SCIENCE\Digital Image Processing\cascade\haarcascade_frontalface_default.xml")
    video_capture = cv2.VideoCapture(0) # turn on camera
    
    while True:
        _ ,frame = video_capture.read() # start real-time recording
        frame = cv2.flip(frame, 1) # let camera behaves like a mirror
        frame = resize(frame, width=1000) # resize window using imutils.resize
        
        gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)

        for (x, y, w, h) in faces_detected:
            cv2.rectangle(frame, (x,y), (x+w,y+h), (255, 0, 0), thickness=5)
            
            roi_gray = gray_img[y:y+w,x:x+h] # cropping region of interest (roi) i.e. face area from image
            roi_gray = cv2.resize(roi_gray, (48, 48))
            img_pixels = image.img_to_array(roi_gray)
            img_pixels = expand_dims(img_pixels, axis = 0)
            img_pixels /= 255

            predictions = model.predict(img_pixels)
            max_index = argmax(predictions[0]) # find max indexed array

            emotions = ('ANGRY', 'DISGUST', 'FEAR', 'HAPPY', 'SAD', 'SURPRISE', 'NEUTRAL')
            predicted_emotion = emotions[max_index]
            cv2.putText(frame, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5)

        # show the output frame
        cv2.imshow("Facial Emotion Analysis (Press 'Q' to Quit Window)", frame)

        # if the `Q` key was pressed, break from the loop (stop the recording)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    video_capture.release() # turn off camera
    del(video_capture) # delete video_capture object
    cv2.destroyAllWindows() # close all cv2 windows

In [9]:
start_camera()